# "THE PRICE IS RIGHT" Capstone Project

This week - build a model that predicts how much something costs from a description, based on a scrape of Amazon data

# Order of play

DAY 1: Data Curation  
DAY 2: Data Pre-processing  
DAY 3: Evaluation, Baselines, Traditional ML  
DAY 4: Deep Learning and LLMs  
DAY 5: Fine-tuning a Frontier Model  

## DAY 5: Fine-tuning a Frontier Model

Now we will use OpenAI's API to fine-tune our own private variant of GPT-4.1-nano

In [1]:
# imports

import os
import re
import json
from dotenv import load_dotenv
from huggingface_hub import login
from openai import OpenAI
from pricer.items  import Item
from pricer.evaluator import evaluate

In [2]:
# environment

LITE_MODE = True

load_dotenv(override=True)
hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


In [4]:
openai = OpenAI()

# Data size

OpenAI recommends fine-tuning with a small population of 50-100 examples

I'm going to go with 20,000 points.

This cost me $3.42 - you should stick with 100 examples and the cost will be minimal!

In [5]:
# OpenAI recommends fine-tuning with populations of 50-100 examples
# But as our examples are very small, I'm suggesting we go with 100 examples (and 1 epoch)


fine_tune_train = train[:100]
fine_tune_validation = val[:50]

In [6]:
len(fine_tune_train)

100

# Step 1

Prepare our data for fine-tuning in JSONL (JSON Lines) format and upload to OpenAI

In [7]:
def messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [
        {"role": "user", "content": message},
        {"role": "assistant", "content": f"${item.price:.2f}"}
    ]

In [8]:
messages_for(fine_tune_train[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \nCategory: Home Hardware  \nBrand: Schlage  \nDescription: A single‑piece oil‑rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \nDetails: Designed for a 4" minimum center‑to‑center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation.'},
 {'role': 'assistant', 'content': '$64.30'}]

In [9]:
# Convert the items into a list of json objects - a "jsonl" string
# Each row represents a message in the form:
# {"messages" : [{"role": "system", "content": "You estimate prices...


def make_jsonl(items):
    result = ""
    for item in items:
        messages = messages_for(item)
        messages_str = json.dumps(messages)
        result += '{"messages": ' + messages_str +'}\n'
    return result.strip()

In [10]:
print(make_jsonl(train[:3]))

{"messages": [{"role": "user", "content": "Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Schlage F59 & 613 Andover Interior Knob (Deadbolt Included)  \nCategory: Home Hardware  \nBrand: Schlage  \nDescription: A single\u2011piece oil\u2011rubbed bronze knob that mounts to a deadbolt for secure, easy interior door use.  \nDetails: Designed for a 4\" minimum center\u2011to\u2011center door prep, it offers a lifetime mechanical and finish warranty and comes ready for quick installation."}, {"role": "assistant", "content": "$64.30"}]}
{"messages": [{"role": "user", "content": "Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Mini Electric Air Duster Fan  \nCategory: Electronics  \nBrand: Kica  \nDescription: Ultra\u2011compact 86,000\u202fRPM electric air duster with 11\u202fm/s wind speed for precise cleaning and inflation.  \nDetails: Powered by a 9.99\u202fWh motor, adjustable in four speed levels, it uses three 

In [11]:
# Convert the items into jsonl and write them to a file

def write_jsonl(items, filename):
    with open(filename, "w") as f:
        jsonl = make_jsonl(items)
        f.write(jsonl)

In [12]:
write_jsonl(fine_tune_train, "jsonl/fine_tune_train.jsonl")

In [13]:
write_jsonl(fine_tune_validation, "jsonl/fine_tune_validation.jsonl")

In [14]:
with open("jsonl/fine_tune_train.jsonl", "rb") as f:
    train_file = openai.files.create(file=f, purpose="fine-tune")

In [15]:
train_file

FileObject(id='file-RW8kreWFuoxRzPy3WqKQWm', bytes=55219, created_at=1770319574, filename='fine_tune_train.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

In [16]:
with open("jsonl/fine_tune_validation.jsonl", "rb") as f:
    validation_file = openai.files.create(file=f, purpose="fine-tune")

In [17]:
validation_file

FileObject(id='file-5qEUvyXmCTeKGUnVVT63E4', bytes=27686, created_at=1770319584, filename='fine_tune_validation.jsonl', object='file', purpose='fine-tune', status='processed', expires_at=None, status_details=None)

https://platform.openai.com/storage/files/

# Step 2

## And now time to Fine-tune!

In [18]:
openai.fine_tuning.jobs.create(
    training_file=train_file.id,
    validation_file=validation_file.id,
    model="gpt-4.1-nano-2025-04-14",
    seed=42,
    hyperparameters={"n_epochs": 1, "batch_size": 1},
    suffix="pricer"
)

FineTuningJob(id='ftjob-LSVjkabRipSUfWtwjxm12Kys', created_at=1770319621, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-OIgYr4OKZdzQl3b0nmya9ngp', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-RW8kreWFuoxRzPy3WqKQWm', validation_file='file-5qEUvyXmCTeKGUnVVT63E4', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, shared_with_openai=False, eval_id=None)

In [19]:
openai.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-LSVjkabRipSUfWtwjxm12Kys', created_at=1770319621, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-OIgYr4OKZdzQl3b0nmya9ngp', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-RW8kreWFuoxRzPy3WqKQWm', validation_file='file-5qEUvyXmCTeKGUnVVT63E4', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, shared_with_openai=False, eval_id=None)], has_more=False, object='list')

In [20]:
job_id = openai.fine_tuning.jobs.list(limit=1).data[0].id

In [21]:
job_id

'ftjob-LSVjkabRipSUfWtwjxm12Kys'

In [22]:
openai.fine_tuning.jobs.retrieve(job_id)

FineTuningJob(id='ftjob-LSVjkabRipSUfWtwjxm12Kys', created_at=1770319621, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1), model='gpt-4.1-nano-2025-04-14', object='fine_tuning.job', organization_id='org-OIgYr4OKZdzQl3b0nmya9ngp', result_files=[], seed=42, status='validating_files', trained_tokens=None, training_file='file-RW8kreWFuoxRzPy3WqKQWm', validation_file='file-5qEUvyXmCTeKGUnVVT63E4', estimated_finish=None, integrations=[], metadata=None, method=Method(type='supervised', dpo=None, reinforcement=None, supervised=SupervisedMethod(hyperparameters=SupervisedHyperparameters(batch_size=1, learning_rate_multiplier='auto', n_epochs=1))), user_provided_suffix='pricer', usage_metrics=None, shared_with_openai=False, eval_id=None)

In [33]:
openai.fine_tuning.jobs.list_events(fine_tuning_job_id=job_id, limit=10).data

[FineTuningJobEvent(id='ftevent-ZA2exriAgrddPMiJapOs3q04', created_at=1770320226, level='info', message='Evaluating model against our usage policies', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-ixDcRdYGSNC6sMfCuXwDYyJJ', created_at=1770320226, level='info', message='New fine-tuned model created', object='fine_tuning.job.event', data={}, type='message'),
 FineTuningJobEvent(id='ftevent-j68FMl27VkGm8RofksLUiUMU', created_at=1770320172, level='info', message='Step 100/100: training loss=1.04, validation loss=2.00, full validation loss=1.24', object='fine_tuning.job.event', data={'step': 100, 'train_loss': 1.0407823324203491, 'valid_loss': 2.001865863800049, 'total_steps': 100, 'full_valid_loss': 1.244896411895752, 'train_mean_token_accuracy': 0.6666666666666666, 'valid_mean_token_accuracy': 0.6666666666666666, 'full_valid_mean_token_accuracy': 0.76}, type='metrics'),
 FineTuningJobEvent(id='ftevent-g5xMGtB397TPgJPsExmUMDkP', created_at=177032

https://platform.openai.com/finetune


# Step 3

Test our fine tuned model

In [34]:
fine_tuned_model_name = openai.fine_tuning.jobs.retrieve(job_id).fine_tuned_model

In [35]:
fine_tuned_model_name

'ft:gpt-4.1-nano-2025-04-14:personal:pricer:D5zRy3j5'

In [36]:
# The prompt

def test_messages_for(item):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}"
    return [
        {"role": "user", "content": message},
    ]

In [37]:
# Try this out

test_messages_for(test[0])

[{'role': 'user',
  'content': 'Estimate the price of this product. Respond with the price, no explanation\n\nTitle: Excess V2 Distortion/Modulation Pedal  \nCategory: Music Pedals  \nBrand: Old Blood Noise  \nDescription: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  \nDetails: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.'}]

In [38]:
# The inference function


def gpt_4__1_nano_fine_tuned(item):
    response = openai.chat.completions.create(
        model=fine_tuned_model_name,
        messages=test_messages_for(item),
        max_tokens=7
    )
    return response.choices[0].message.content

In [39]:
print(test[0].price)
print(gpt_4__1_nano_fine_tuned(test[0]))

219.0
$194.35


In [40]:
evaluate(gpt_4__1_nano_fine_tuned, test)

  0%|          | 0/200 [00:00<?, ?it/s]

$0 $19 $9 $34 $40 $143 $53 $55 $6 $467 $325 $80 $20 $11 $16 $4 $151 $3 $27 $29 $56 $86 $7 $26 $188 $264 $200 $2 $301 $62 $46 $18 $82 $60 $115 $571 $105 $27 $6 $21 $152 $43 $20 $25 $62 $10 $82 $8 $104 $68 $26 $117 $397 $14 $143 $202 $13 $50 $139 $1 $106 $33 $43 $25 $29 $15 $194 $381 $130 $13 $12 $44 $57 $9 $10 $14 $56 $0 $11 $11 $48 $11 $0 $89 $173 $10 $359 $6 $0 $22 $13 $207 $14 $59 $11 $36 $8 $52 $105 $249 $44 $71 $47 $106 $193 $118 $2 $342 $39 $29 $33 $7 $34 $128 $34 $8 $43 $4 $4 $8 $10 $1143 $7 $483 $97 $317 $3 $81 $81 $44 $42 $56 $13 $0 $95 $0 $9 $21 $25 $138 $14 $106 $108 $11 $155 $98 $59 $504 $104 $3 $2 $173 $3 $40 $8 $140 $405 $31 $5 $47 $73 $16 $16 $7 $205 $9 $448 $22 $4 $8 $11 $2 $21 $62 $46 $369 $0 $198 $11 $96 $205 $4 $40 $10 $34 $26 $56 $12 $20 $1 $1 $78 $34 $176 $70 $246 $0 $90 $13 $9 

In [ ]:
# 96.58 - mini 200
# 79.29 - mini 2000
# 82.26 - nano 2000
# 67.75 - nano 20,000